# **Algoritmo heuristico - Buscaminas**

**Autores:** Felix Barba y Jesús Quiróz Rojas

**Instalar librerias**

Se instalan las librerias de opencv, numpy, pyautogui y matplotlib

In [6]:
# @title Instalacion de librerias
!pip install pillow opencv-python numpy pyautogui matplotlib


## Procesamiento de Imágenes para Buscaminas

Este código está diseñado para capturar, procesar y analizar imágenes de un tablero del juego Buscaminas utilizando técnicas de visión por computadora. A continuación se describe cada parte del código:

### Funciones Principales

1. **Captura de Pantalla (`take_screenshot`)**
   - **Propósito:** Captura una imagen de la pantalla en el momento en que se llama a la función.
   - **Cómo Funciona:** Usa `pyautogui.screenshot()` para tomar una captura y la guarda como `captura_buscaminas.png`.

2. **Mostrar Imagen (`show_image`)**
   - **Propósito:** Muestra una imagen en una ventana emergente.
   - **Cómo Funciona:** Utiliza `cv2.imshow` para mostrar la imagen y `cv2.waitKey(0)` para mantener la ventana abierta hasta que se presione una tecla.

3. **Detectar Región del Tablero (`detect_board_region`)**
   - **Propósito:** Identifica y recorta el área del tablero en la imagen capturada.
   - **Cómo Funciona:** Convierte la imagen a escala de grises, aplica desenfoque y detección de bordes para encontrar contornos. Filtra los contornos que tienen forma de rectángulo y son lo suficientemente grandes para ser considerados como el tablero del juego.

4. **Dividir Tablero en Celdas (`split_board_into_cells`)**
   - **Propósito:** Divide la región del tablero detectada en una cuadrícula de celdas.
   - **Cómo Funciona:** Calcula la altura y el ancho de cada celda dividiendo la imagen del tablero por el número de filas y columnas (9x9). Extrae cada celda individualmente.

5. **Clasificar Celda (`classify_cell`)**
   - **Propósito:** Clasifica las celdas como descubiertas o cubiertas basándose en su color promedio.
   - **Cómo Funciona:** Convierte la celda a RGB y calcula el color promedio. Utiliza umbrales predefinidos para distinguir entre celdas descubiertas (azul) y cubiertas (blanco).

6. **Detectar Número en Celda (`detect_number_in_cell`)**
   - **Propósito:** Identifica números en celdas descubiertas usando técnicas de coincidencia de plantillas.
   - **Cómo Funciona:** Convierte la celda a escala de grises, aplica umbralización binaria y usa `cv2.matchTemplate` para comparar la celda con plantillas de números predefinidos (1, 2, 3). Devuelve el número detectado basado en la mejor coincidencia.

7. **Convertir Celdas en Matriz (`cells_to_board_matrix`)**
   - **Propósito:** Convierte el conjunto de celdas en una matriz que representa el tablero del juego.
   - **Cómo Funciona:** Clasifica cada celda y, para las celdas descubiertas, detecta el número en ellas. Devuelve una matriz con valores que indican celdas descubiertas vacías (`-1`), celdas cubiertas (`0`), o números detectados (`1`, `2`, `3`).

### Función Principal

- **Propósito:** Coordina el flujo de trabajo para capturar una imagen, procesar el tablero y generar una matriz del estado del tablero.
- **Cómo Funciona:**
  - Carga las plantillas de números.
  - Captura una imagen de la pantalla.
  - Detecta la región del tablero en la imagen.
  - Divide el tablero en celdas.
  - Convierte las celdas en una matriz representativa del tablero.
  - Devuelve la matriz resultante.

### Ejecución

El código se ejecuta llamando a la función `main()`, que realiza todas las tareas descritas y finalmente devuelve una matriz que representa el tablero del Buscaminas.



In [7]:
# @title Procesamiento de imagenes
import cv2
import numpy as np
import pyautogui

# Función para tomar una captura de pantalla
def take_screenshot():
    screenshot = pyautogui.screenshot()
    screenshot_path = 'captura_buscaminas.png'
    screenshot.save(screenshot_path)
    return screenshot_path

# Función para mostrar la imagen
def show_image(image, title="Imagen"):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Función para detectar el área del tablero mediante contornos
def detect_board_region(img, rows=9, cols=9):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Encontrar contornos
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar los contornos que tienen forma de cuadrado o rectángulo
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        area = cv2.contourArea(contour)
        if len(approx) == 4 and area > 1000:  # Rectángulo grande
            x, y, w, h = cv2.boundingRect(approx)
            board_region = img[y:y+h, x:x+w]
            return board_region

    return None

# Función para dividir el tablero en celdas (9x9)
def split_board_into_cells(board_img, rows=9, cols=9):
    cell_height = board_img.shape[0] // rows
    cell_width = board_img.shape[1] // cols

    cells = []
    for i in range(rows):
        row = []
        for j in range(cols):
            cell = board_img[i*cell_height:(i+1)*cell_height, j*cell_width:(j+1)*cell_width]
            row.append(cell)
        cells.append(row)
    return cells

# Función para clasificar una celda basada en el color (azul = descubierta, blanco = cubierta)
def classify_cell(cell):
    # Convertir la celda a RGB
    rgb_cell = cv2.cvtColor(cell, cv2.COLOR_BGR2RGB)

    # Obtener el promedio de color de la celda
    mean_color = np.mean(rgb_cell, axis=(0, 1))  # [R, G, B]

    # Definir umbrales para clasificación basado en los promedios de color
    blue_threshold = (21,92,158)  # Umbral para detectar azul (descubierta)
    white_threshold = (150,190,230)  # Umbral para detectar blanco (cubierta)

    # Clasificación basada en el color promedio
    if (mean_color[0] < blue_threshold[0] and
        mean_color[1] < blue_threshold[1] and
        mean_color[2] < blue_threshold[2]):
        return -1  # Descubierta (azul, sin número)
    elif (mean_color[0] > white_threshold[0] and
          mean_color[1] > white_threshold[1] and
          mean_color[2] > white_threshold[2]):
        return 0  # Cubierta (blanco)
    else:
        return -1  # Si no se clasifica claramente, se asume descubierta

# Función para identificar números en las celdas descubiertas usando template matching
def detect_number_in_cell(cell, templates):
    gray_cell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)

    # Aplicar umbralización binaria para mejorar la visibilidad de los números
    _, binary_cell = cv2.threshold(gray_cell, 127, 255, cv2.THRESH_BINARY)

    detected_number = -1
    max_val = 0  # Para almacenar la mejor coincidencia encontrada

    # Iterar sobre cada plantilla y aplicar template matching
    for number, template in templates.items():
        result = cv2.matchTemplate(binary_cell, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_loc_val, min_loc, max_loc = cv2.minMaxLoc(result)

        # Si la coincidencia actual es mejor que las anteriores, actualizamos
        if max_loc_val > max_val:
            max_val = max_loc_val
            detected_number = number

    # Definir un umbral de coincidencia para asegurar que sea una buena coincidencia
    if max_val >= 0.3:  # Reducido para mejorar coincidencias
        return detected_number
    else:
        return -1  # Si no se encuentra una coincidencia suficiente, retorna -1

# Función para convertir las celdas en una matriz con los números, -1 (descubierta vacía) y 0 (cubierta)
def cells_to_board_matrix(cells, templates):
    matrix = []
    for row in cells:
        matrix_row = []
        for cell in row:
            classification = classify_cell(cell)

            if classification == -1:  # Celda descubierta, verificar si tiene número
                detected_number = detect_number_in_cell(cell, templates)
                if detected_number > 0:
                    matrix_row.append(detected_number)  # Número detectado
                else:
                    matrix_row.append(-1)  # Celda vacía
            else:
                matrix_row.append(classification)  # Celda cubierta (0)
        matrix.append(matrix_row)
    return matrix

# Función principal
def main():
    # Cargar las plantillas de números (1, 2 y 3)
    templates = {
        1: cv2.imread('1.png', cv2.IMREAD_GRAYSCALE),
        2: cv2.imread('2.png', cv2.IMREAD_GRAYSCALE),
        3: cv2.imread('3.png', cv2.IMREAD_GRAYSCALE)
    }

    # Verificar si las plantillas se cargaron correctamente
    for key, template in templates.items():
        if template is None:
            print(f"Error al cargar la plantilla para el número {key}.")
            return

    screenshot_path = take_screenshot()
    img = cv2.imread(screenshot_path)

    if img is None:
        print("Error al cargar la captura de pantalla.")
        return

    # Detectar el área del tablero
    board_region = detect_board_region(img, rows=9, cols=9)
    if board_region is None:
        print("No se pudo detectar el tablero.")
        return

    # Mostrar la región detectada
    #show_image(board_region, "Región del tablero detectada")

    # Dividir el tablero en celdas 9x9
    cells = split_board_into_cells(board_region, rows=9, cols=9)

    # Convertir las celdas en una matriz de números, -1 (descubierta vacía) y 0 (cubierta)
    board_matrix = cells_to_board_matrix(cells, templates)

    # Imprimir la matriz

    return board_matrix

# Ejecutar el código
tablero = main()

tablero = np.array(tablero)

**Algoritmo de busqueda**



## Análisis y Procesamiento del Tablero de Buscaminas

Este código está diseñado para analizar y procesar un tablero de Buscaminas utilizando matrices para gestionar la información sobre las celdas descubiertas y cubiertas, así como calcular el riesgo asociado a cada celda. A continuación se describe el funcionamiento de cada sección del código:

### Inicialización

1. **Importación de Bibliotecas**
   - **`import numpy as np`**: Se importa la biblioteca NumPy para la manipulación de matrices y operaciones matemáticas.

2. **Carga del Tablero**
   - **`tablero = main()`**: Llama a la función `main()` (Seccion de procesamiento de imagenes) para obtener el tablero del juego en forma de matriz.
   - **`tablero = np.array(tablero)`**: Convierte el tablero a una matriz NumPy para facilitar el procesamiento.

3. **Inicialización de Matrices**
   - **`auxiliar`, `restriction`, `risk`**: Se crean matrices de ceros con tamaño 9x9 para almacenar información sobre el tablero, las restricciones y el riesgo asociado a cada celda.

### Funciones

1. **`markCell(board, i, j, auxiliar, restriction, risk)`**
   - **Propósito:** Actualiza las matrices `auxiliar`, `restriction` y `risk` basándose en la información de una celda en el tablero.
   - **Cómo Funciona:** Para cada celda en el tablero, se examinan las celdas adyacentes. Si una celda adyacente está cubierta (`board[ni][nj] == 0`), se actualizan las matrices `auxiliar`, `restriction` y `risk` con base en el valor de la celda actual (`board[i][j]`).

2. **`check_board(board, auxiliar, restriction, risk)`**
   - **Propósito:** Procesa todo el tablero para actualizar las matrices `auxiliar`, `restriction` y `risk`.
   - **Cómo Funciona:** Recorre cada celda del tablero. Si una celda contiene un número positivo, se llama a `markCell` para actualizar las matrices y luego se ajusta el valor en `auxiliar`.

3. **`tachZero(auxiliar)`**
   - **Propósito:** Reemplaza los ceros en la matriz `auxiliar` por `-1` para indicar celdas sin información útil.
   - **Cómo Funciona:** Utiliza la indexación booleana de NumPy para hacer esta sustitución en toda la matriz.

4. **`restrictionNormalization(auxiliar, restriction)`**
   - **Propósito:** Normaliza los valores en la matriz `auxiliar` dividiéndolos por los valores en la matriz `restriction`.
   - **Cómo Funciona:** Usa la función `np.where` para dividir `auxiliar` por `restriction` sólo donde `restriction` no es cero, y maneja la división por cero sin errores.

### Procesamiento y Resultados

1. **Procesamiento del Tablero**
   - **`check_board(tablero, auxiliar, restriction, risk)`**: Llama a la función `check_board` para procesar el tablero.
   - **`tachZero(auxiliar)`**: Reemplaza ceros en la matriz `auxiliar`.

2. **Impresión de Resultados**
   - **Matrices de Restricciones y Riesgo:** Se imprimen las matrices `restriction` y `risk` para revisar la información calculada.
   - **Valores en el Rango [1, 100]:** Se identifican los valores mínimos en el rango [1, 100] de la matriz `auxiliar`.
   - **Coordenadas Mínimas y Riesgo Asociado:**
     - **`coordenadas_minimos`**: Identifica las coordenadas con el valor mínimo en `auxiliar`.
     - **`valores_riesgo_minimos`**: Obtiene el riesgo asociado a las coordenadas mínimas.
     - **`coordenadas_risk_min`**: Encuentra las coordenadas con el riesgo mínimo.
     - **Impresión de Coordenadas y Riesgo:** Imprime las coordenadas con el valor mínimo y su riesgo asociado.

3. **Valor Máximo en la Matriz de Restricciones**
   - **`max_valor_restriction`**: Calcula e imprime el valor máximo en la matriz `restriction`.

### Ejecución

El código se ejecuta llamando a la función `main()`, procesando el tablero del Buscaminas y generando matrices de información relevante. El análisis proporciona información útil sobre el estado del tablero y el riesgo asociado a cada celda, lo que puede ayudar en la toma de decisiones durante el juego.



In [9]:
# @title Start
import numpy as np

tablero = main()
tablero = np.array(tablero)
print('Tablero ac')
print(tablero)

# Inicialización de matrices
auxiliar = np.zeros((9, 9))
restriction = np.zeros((9, 9))
risk = np.zeros((9,9))

# Función para marcar celdas basadas en restricciones
def markCell(board, i, j, auxiliar, restriction, risk):
    # Definir las direcciones en las que se pueden mover
    directions = [
        (0, 1),   # derecha
        (1, 0),   # abajo
        (1, 1),   # diagonal inferior derecha
        (-1, 0),  # arriba
        (0, -1),  # izquierda
        (-1, -1), # diagonal superior izquierda
        (-1, 1),  # diagonal superior derecha
        (1, -1)   # diagonal inferior izquierda
    ]

    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < 9 and 0 <= nj < 9 and board[ni][nj] == 0:
            auxiliar[ni][nj] += board[i][j]
            restriction[ni][nj] += 1
            risk[ni,nj]= (auxiliar[ni][nj]//restriction[ni][nj])

# Función para procesar el tablero
def check_board(board, auxiliar, restriction, risk):
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0 and board[i][j] != -1:
                markCell(board, i, j, auxiliar, restriction, risk)
                auxiliar[i][j] = 100 + board[i][j]

# Función para reemplazar ceros en la matriz auxiliar
def tachZero(auxiliar):
    auxiliar[auxiliar == 0] = -1

# Normalización de la matriz auxiliar según las restricciones
def restrictionNormalization(auxiliar, restriction):
    with np.errstate(divide='ignore', invalid='ignore'):
        auxiliar = np.where(restriction != 0, auxiliar / restriction, auxiliar)
    return auxiliar



# Llamada a las funciones principales
check_board(tablero, auxiliar, restriction, risk)
tachZero(auxiliar)



print("Matriz de restricciones:")
print(restriction)

print('Matriz de Riesgo')
print('')
print(risk)
# Encontrar los valores en el rango de [1, 100]
valores_en_rango = auxiliar[(auxiliar >= 1) & (auxiliar <= 100)]
valores_risk_min = risk[(risk>=1)]
def format_coordinates(coords):
    return [tuple(map(int, coord)) for coord in coords]

# Valor mínimo dentro del rango y coordenadas correspondientes
if len(valores_en_rango) > 0:
    min_valor = np.min(valores_en_rango)
    coordenadas_minimos = np.argwhere(auxiliar == min_valor)
    valores_riesgo_minimos = [risk[tuple(coord)] for coord in coordenadas_minimos]

    min_valor_risk = np.min(valores_risk_min)
    risk_min = np.argwhere(risk==min_valor_risk)

    coordenadas_minimos_set = set(tuple(c) for c in coordenadas_minimos)
    coordenadas_risk_min_set = set(tuple(c) for c in risk_min)
    print('Coordenadas minimas pero con alto riesgo')
    formatted_coords = format_coordinates(coordenadas_minimos_set)

    for coord, riesgo in zip(formatted_coords, valores_riesgo_minimos):
        print(f"Coordenada: {coord}, Riesgo: {riesgo:.2f}")


else:
    print("No se encontraron valores en el rango [1, 100].")

# Valor máximo en la matriz de restricciones
max_valor_restriction = np.max(restriction)
print(f"El valor máximo en la matriz de restricciones es: {max_valor_restriction}")

Tablero ac
[[ 2  0  2 -1 -1  1  0  0  0]
 [ 0  0  2 -1 -1  1  1  2  1]
 [ 2  2  1  1  1  1 -1 -1 -1]
 [-1  2 -1  0  0  1 -1 -1 -1]
 [-1  2  0  2  1  1 -1 -1 -1]
 [-1  2  1  1 -1 -1  1  1  1]
 [-1 -1 -1 -1 -1  1  2  0  0]
 [-1  2  1  1 -1  1  0  0  0]
 [-1  2  0  1 -1  1  2  0  0]]
Matriz de restricciones:
[[0. 3. 0. 0. 0. 0. 4. 3. 2.]
 [3. 6. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 5. 7. 0. 0. 0. 0.]
 [0. 0. 6. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 4. 2.]
 [0. 0. 0. 0. 0. 0. 5. 2. 0.]
 [0. 0. 5. 0. 0. 0. 0. 1. 0.]]
Matriz de Riesgo

[[0. 2. 0. 0. 0. 0. 1. 1. 1.]
 [2. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 2. 0.]
 [0. 0. 1. 0. 0. 0. 0. 2. 0.]]
Coordenadas minimas pero con alto riesgo
Coordenada: (8, 7), Riesgo: 1.00
Coordenada: (6, 8), Riesgo: 2.00
El valor máximo en la matr